In [ ]:
# Verisetimi Google Drive'a yükledim. Verisetimi almak için Google Drive'a bağlanıyorum.
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'İNDİRİLEBİLİR İÇERİKLER'  'My Drive'	    yolov3
 images		    MUSTAFA		      'ÖDEV PROJELER.rar'


In [ ]:
# Eğitimimi Darknet aracılığıyla yapacağım için Colab'a Darkneti yüklüyorum.
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15521, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 15521 (delta 0), reused 4 (delta 0), pack-reused 15514
Receiving objects: 100% (15521/15521), 14.17 MiB | 10.66 MiB/s, done.
Resolving deltas: 100% (10413/10413), done.


In [ ]:
# Eğitimi hızlı bir şekilde yapmak için Colab'ın bize sunduğu GPU'yu kullanacağım.
# Colabın bana verdiği GPU'yu Darknet için derliyorum. 
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (

In [ ]:
# Yolov3'ün cfg dosyasını yüklüyorum.
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Yüklemiş olduğum yolov3 cfg dosyasını kendi değerlerime göre düzenliyorum.
# batch değerini 64 yapıyorum, yani bir seferde 64 örnek işlenecek şekilde ayarlıyorum.
# Belleği iyi kullanmak için subdivisions değerini 16 yapıyorum, yani 16 parça halinde çalışacak.
# Eğitimin 2000 adım sürmesini istiyorum, max_batches değerini 2000 yapıyorum.
# Cfg'nin belirli satırlarında bulunan classes ve filters değerlerini kendim için değiştiriyorum.
# Ben sadece kus nesnesi için çalışacağım bu sebeple classes değerlerini 1, filters değerlerinide 18 yapıyorum.
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Eğitimden sonra oluşacak olan .weights dosyasının Drive'a kaydedileceği yeri ayarlıyorum.
!mkdir "/mydrive/yolov3"

In [ ]:
# Yoloda kullanılan .names isimli dosya içerisine kus nesnesini ekliyorum.
# İçerisinde classes=1, train.txt, test.txt, obj.names ve backup dosyalarının yollarını barındıran obj.data dosyasını oluşturuyorum.
!echo "kus" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Darknet ağırlıklarını getiriyorum.
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-05-27 16:15:00--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  17.7MB/s    in 10s     

2023-05-27 16:15:11 (15.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
# Google Drive'a yüklemiş olduğum images.zip verisetimi data/obj klasörüne çıkarıyorum.
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/2007_000363.jpg  
  inflating: data/obj/2007_000363.txt  
  inflating: data/obj/2007_000645.jpg  
  inflating: data/obj/2007_000645.txt  
  inflating: data/obj/2007_001289.jpg  
  inflating: data/obj/2007_001289.txt  
  inflating: data/obj/2007_003349.jpg  
  inflating: data/obj/2007_003349.txt  
  inflating: data/obj/2007_004143.jpg  
  inflating: data/obj/2007_004143.txt  
  inflating: data/obj/2007_005264.jpg  
  inflating: data/obj/2007_005264.txt  
  inflating: data/obj/2007_007481.jpg  
  inflating: data/obj/2007_007481.txt  
  inflating: data/obj/2007_009607.jpg  
  inflating: data/obj/2007_009607.txt  
  inflating: data/obj/2007_009759.jpg  
  inflating: data/obj/2007_009759.txt  
  inflating: data/obj/2007_009817.jpg  
  inflating: data/obj/2007_009817.txt  
  inflating: data/obj/2008_000097.jpg  
  inflating: data/obj/2008_000097.txt  
  inflating: data/obj/2008_000103.jpg  
  inflating: data/obj/2008_000103.txt  
  i

In [ ]:
# .jpg uzantılı resimlerimizin yolunu almak için, glob modülünü ekliyorum.
# Aldığım dosya yollarını gorselYollari adlı listeme aktarıyorum ve ekranda bu yolları gösteriyorum.
import glob
gorselYollari = glob.glob("data/obj/*.jpg")
print(gorselYollari)

['data/obj/2008_006924.jpg', 'data/obj/2008_007501.jpg', 'data/obj/2007_003349.jpg', 'data/obj/2008_008292.jpg', 'data/obj/2008_001020.jpg', 'data/obj/2008_007854.jpg', 'data/obj/2008_003580.jpg', 'data/obj/2009_000218.jpg', 'data/obj/2008_002471.jpg', 'data/obj/2007_009759.jpg', 'data/obj/2008_007477.jpg', 'data/obj/2009_000503.jpg', 'data/obj/2008_003694.jpg', 'data/obj/2007_009607.jpg', 'data/obj/2008_005618.jpg', 'data/obj/2008_008022.jpg', 'data/obj/2009_002890.jpg', 'data/obj/2008_007120.jpg', 'data/obj/2008_007434.jpg', 'data/obj/2008_004296.jpg', 'data/obj/2008_007593.jpg', 'data/obj/2008_008287.jpg', 'data/obj/2008_008544.jpg', 'data/obj/2008_004425.jpg', 'data/obj/2008_008595.jpg', 'data/obj/2009_000849.jpg', 'data/obj/2008_006715.jpg', 'data/obj/2009_004141.jpg', 'data/obj/2008_004103.jpg', 'data/obj/2008_008284.jpg', 'data/obj/2009_001397.jpg', 'data/obj/2008_007188.jpg', 'data/obj/2009_001375.jpg', 'data/obj/2009_001972.jpg', 'data/obj/2008_005843.jpg', 'data/obj/2008_0024

In [ ]:
# train.txt adında bir dosya oluşturuyorum ve içine gorselYollari listesindeki değerleri yazdırıyorum.
file = open("data/train.txt", "w") 
file.write("\n".join(gorselYollari)) 
file.close() 

In [ ]:
# Oluşturduğum obj.data ve yolov3_training.cfg dosyaları ile verisetimi eğitmeye başlıyorum.
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Görüntülenen çıkış son 5000 satıra kısaltıldı.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
 total_bbox = 143274, rewritten_bbox = 0.013261 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.876731), count: 4, class_loss = 0.005870, iou_loss = 0.092358, total_loss = 0.098228 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000002, iou_loss = 0.000000, total_loss = 0.000002 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 143278, rewritten_bbox = 0.013261 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.817040), count: 4, class_loss = 0.230555, iou_loss = 0.128012, total_loss = 0.358568 
v3 (mse l